Automatic Document Clustering [Dataset: newsgroups5.zip]
For this problem you will use a different subset of the 20 Newsgroup data set that you used in Assignment 2  (see the description of the full dataset). The subset for this assignment includes 2,500 documents (newsgroup posts), each belonging to one of 5 categories windows (0), crypt (1), christian (2), hockey (3), forsale (4). The documents are represented by 9328 terms (stems). The dictionary (vocabulary) for the data set is given in the file "terms.txt" and the full term-by-document matrix is given in "matrix.txt" (comma separated values). The actual category labels for the documents are provided in the file "classes.txt". Your goal in this assignment is to perform clustering on the documents and compare the clusters to the actual categories.

Your tasks in this problem are the following [Note: for the clustering part of this assignment you should use the kMeans module form Ch. 10 of MLA (use the version provided here as it includes some corrections to the book version). You may also use Pandas and other modules from scikit-learn that you may need for preprocessing or evaluation.]

In [1]:
import pandas as pd 
import numpy as np

### a

Create your own distance function that, instead of using Euclidean distance, uses Cosine similarity. This is the distance function you will use to pass to the kMeans function.

In [2]:
def cosine_sim(v1,v2):
    v1_norm = np.linalg.norm(v1)
    v2_norm = np.linalg.norm(v2)
    cos_sim = (np.dot(v1,v2))/(v1_norm * v2_norm)
    distance = 1 - cos_sim
    return distance


### b
Load the data set [Note: the data matrix provided has terms as rows and documents as columns. Since you will be clustering documents, you'll need to take the transpose of this matrix so that your main data matrix is a document x term matrix. In Numpy, you may use the ".T" operation to obtain the transpose.] Then, split the data set (the document x term matrix) and set aside 20% for later use (see below). Use the 80% segment for clustering in the next part. The 20% portion must be a random subset.

In [3]:
pwd

u'C:\\Users\\Taha'

In [4]:
import os 
os.chdir('E:/OneDrive/MSPA/CSC 478/Assignment/Assignment 3/newsgroups5')

In [5]:
df_terms = pd.read_csv('terms.txt', na_values = ['?'], dtype = str, header = None)
print df_terms.shape
df_terms.head()

(9328, 1)


,0
0,aa
1,aargh
2,aaron
3,aaronc
4,ab


In [6]:
df_matrix = pd.read_csv('matrix.txt', na_values = ['?'], dtype = int, header = None)
print df_matrix.shape
df_matrix.head()
DT_mat = df_matrix.transpose()
print DT_mat.shape
DT_mat.head()

(9328, 2500)
(2500, 9328)


,0,1,2,3,4,5,6,7,8,9,...,9318,9319,9320,9321,9322,9323,9324,9325,9326,9327
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df_class = pd.read_csv('classes.txt', skiprows= [0], na_values = ['?'], index_col= 0, delim_whitespace= True, header =  None)
print df_class.shape
df_class.head()

(2500, 1)


,1
0,
0,0
1,1
2,1
3,1
4,2


In [8]:
from sklearn.cross_validation import train_test_split
train_data, test_data, train_label, test_label = train_test_split(DT_mat, df_class, test_size = 0.2, random_state = 44)

In [9]:
print 'The shape of Train Data is ', train_data.shape
print 'The shape of Train Label is ', train_label.shape
print 'The shape of Test Data is ', test_data.shape
print 'The shape of Test Label is ', test_label.shape


The shape of Train Data is  (2000, 9328)
The shape of Train Label is  (2000, 1)
The shape of Test Data is  (500, 9328)
The shape of Test Label is  (500, 1)


### c

As in the case of Assignment 2, transform the term-frequencies to tfxidf values. Be sure to maintain DF values for each of the terms in the dictionary. [Note: if you run into problems due to limited computational resources, you may prune the data by removing all terms with low DF values, e.g., terms that appear in less than 10 documents. Be sure to maintain the correspondence between the dictionary terms and the matrix rows.]

In [10]:
train_arr = np.array(train_data)
test_arr = np.array(test_data)
print "The shape of Train Array is : ", train_arr.shape
print "The Shape of Test Array is : ", test_arr.shape

The shape of Train Array is :  (2000L, 9328L)
The Shape of Test Array is :  (500L, 9328L)


In [11]:
num_Doc_train = len(train_arr)
print "The Number of Docs in Training Data is : ", num_Doc_train

num_Terms_train  = len(train_arr[1])
print "The Number of Terms in Training Data is : ", num_Terms_train

num_Doc_test = len(test_arr)
print "The Number of Docs in Testing Data is : ", num_Doc_test

The Number of Docs in Training Data is :  2000
The Number of Terms in Training Data is :  9328
The Number of Docs in Testing Data is :  500


In [12]:
Doc_Freq = np.array([(train_arr != 0).sum(0)])
print 'The Documents Frequency Matrix in Training Data is', Doc_Freq

Term_Freq = train_arr.sum(axis = 0)
print 'The Term Freq Matrix in Training Data is', Term_Freq

The Documents Frequency Matrix in Training Data is [[ 8  4 17 ...,  3  2  2]]
The Term Freq Matrix in Training Data is [14  4 73 ..., 11  5  6]


In [13]:
train_matrix = np.ones(np.shape(train_arr), dtype =float) *num_Doc_train
print 'The Shape of Training Matrix is ', train_matrix.shape
test_matrix = np.ones(np.shape(test_arr), dtype = float) * num_Doc_train
print 'The Shape of Testing Matrix is ', test_matrix.shape

The Shape of Training Matrix is  (2000L, 9328L)
The Shape of Testing Matrix is  (500L, 9328L)


In [14]:
train_IDF = np.log2(np.divide(train_matrix, Doc_Freq))
test_IDF = np.log2(np.divide(test_matrix, Doc_Freq))

print 'The IDF for Train is: \n', train_IDF[0:5]
print 'The IDF for Test is: \n', test_IDF[0:5]

The IDF for Train is: 
[[ 7.96578428  8.96578428  6.87832144 ...,  9.38082178  9.96578428
   9.96578428]
 [ 7.96578428  8.96578428  6.87832144 ...,  9.38082178  9.96578428
   9.96578428]
 [ 7.96578428  8.96578428  6.87832144 ...,  9.38082178  9.96578428
   9.96578428]
 [ 7.96578428  8.96578428  6.87832144 ...,  9.38082178  9.96578428
   9.96578428]
 [ 7.96578428  8.96578428  6.87832144 ...,  9.38082178  9.96578428
   9.96578428]]
The IDF for Test is: 
[[ 7.96578428  8.96578428  6.87832144 ...,  9.38082178  9.96578428
   9.96578428]
 [ 7.96578428  8.96578428  6.87832144 ...,  9.38082178  9.96578428
   9.96578428]
 [ 7.96578428  8.96578428  6.87832144 ...,  9.38082178  9.96578428
   9.96578428]
 [ 7.96578428  8.96578428  6.87832144 ...,  9.38082178  9.96578428
   9.96578428]
 [ 7.96578428  8.96578428  6.87832144 ...,  9.38082178  9.96578428
   9.96578428]]


In [15]:
TDIDF_train = train_arr * train_IDF
print "The Shape of Training Data's TFIDF is: ", TDIDF_train.shape

TDIDF_test = test_arr * test_IDF
print "The Shape of Testing Data's TFIDF is: ", TDIDF_test.shape

The Shape of Training Data's TFIDF is:  (2000L, 9328L)
The Shape of Testing Data's TFIDF is:  (500L, 9328L)


In [16]:
TDIDF_test = np.nan_to_num(TDIDF_test)
TDIDF_train = np.nan_to_num(TDIDF_train)

### d
Perform Kmeans clustering on the training data. Write a function to display the top N terms in each cluster along with the cluster DF values for each term, percentage of docs in the cluster in which the terms appear, and the size of the cluster. Sort the terms in decreasing order of the DF percentage. [Extra Credit: use your favorite third party tool, ideally with a Python based API, to create a word cloud for each cluster based on the in-cluster DF values.]

In [17]:
def randCent(dataSet, k):
    n = shape(dataSet)[1]
    centroids = zeros((k,n), dtype=float)
    for j in range(n): #create random cluster centers
        minJ = min(dataSet[:,j])
        rangeJ = float(max(dataSet[:,j]) - minJ)
        centroids[:,j] = minJ + rangeJ * random.rand(k)
    return centroids 

In [18]:
from numpy import *
def kMeans(dataSet, k, distMeas=cosine_sim, createCent=randCent):
    m = shape(dataSet)[0]
    clusterAssment = zeros((m,2))#create mat to assign data points 
                                      #to a centroid, also holds SE of each point
    centroids = createCent(dataSet, k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):#for each data point assign it to the closest centroid
            minDist = inf; minIndex = -1
            for j in range(k):
                distJI = distMeas(centroids[j,:],dataSet[i,:])
                if distJI < minDist:
                    minDist = distJI; minIndex = j
            if clusterAssment[i,0] != minIndex: clusterChanged = True
            clusterAssment[i,:] = minIndex,minDist**2
        # print centroids
        for cent in range(k):#recalculate centroids
            ptsInClust = dataSet[nonzero(clusterAssment[:,0]==cent)[0]] #get all the point in this cluster - 
            #Note: this was incorrect in the original distribution.
            if(len(ptsInClust)!=0):
                centroids[cent,:] = mean(ptsInClust, axis=0) 
                #assign centroid to mean - Note condition was added 10/28/2013
    return centroids, clusterAssment

In [19]:
center, cluster = kMeans(TDIDF_train,5)

In [23]:
df_TDIDF_train = pd.DataFrame(TDIDF_train)
df_TDIDF_test = pd.DataFrame(TDIDF_test)


In [30]:
from __future__ import division
def terms_clust(train_data,terms_data, k,N ):
    train_mat = np.array(train_data)
    center, cluster = kMeans(train_mat, k,distMeas=cosine_sim)
    df_clust = pd.DataFrame(cluster,columns = ['Cluster', 'Distance'])
    df_cent = pd.DataFrame(center)
    
    for i in range(k):
        c = df_clust[df_clust['Cluster'] == i]
        print 'The Size of Cluster ', i, 'is ', len(c)
        doc_cluster = train_data.ix[c.index]
        doc_array =  np.array(doc_cluster)
        DF = np.array([(doc_array.T!=0).sum(1)]).T
        tot_doc = len(doc_cluster)
        df_DF = pd.DataFrame(DF, columns = ['Count'])
        terms_arr = np.array(terms_data)
        df_DF['Terms'] = terms_arr
        sorted_DF = df_DF.sort_values('Count', ascending= False)
        Ser = pd.Series(sorted_DF.Count)
        Ser = Ser[:N]
        terms =df_terms.ix[Ser.index]
        for x in range(len(Ser)):
            
            print ' The Term "', terms.iloc[x][0], '" appears a total of "', Ser.iloc[x],'" times and the Percentage in Document Frequency is "', Ser.iloc[x]/tot_doc, '"'      

In [32]:
terms_clust(df_TDIDF_train,df_terms, 5, 10)

The Size of Cluster  0 is  378
 The Term " sale " appears a total of " 235 " times and the Percentage in Document Frequency is " 0.621693121693 "
 The Term " email " appears a total of " 144 " times and the Percentage in Document Frequency is " 0.380952380952 "
 The Term " pleas " appears a total of " 127 " times and the Percentage in Document Frequency is " 0.335978835979 "
 The Term " offer " appears a total of " 109 " times and the Percentage in Document Frequency is " 0.28835978836 "
 The Term " ship " appears a total of " 95 " times and the Percentage in Document Frequency is " 0.251322751323 "
 The Term " sell " appears a total of " 94 " times and the Percentage in Document Frequency is " 0.248677248677 "
 The Term " includ " appears a total of " 89 " times and the Percentage in Document Frequency is " 0.23544973545 "
 The Term " interest " appears a total of " 87 " times and the Percentage in Document Frequency is " 0.230158730159 "
 The Term " on " appears a total of " 84 " tim

### e

Using the cluster assignments from Kmeans clustering, compare your 5 clusters to the 5 pre-assigned classes by computing the Completeness and Homogeneity values.

In [33]:
from sklearn.metrics import completeness_score, homogeneity_score

center, cluster = kMeans(TDIDF_train,5)

In [34]:
train_lab_arr = np.array(train_label)
new_df = pd.DataFrame(train_lab_arr, columns = ['Classes_Label'])
clust_df = pd.DataFrame(cluster,index=df_TDIDF_train.index,copy=True, columns = ['Cluster', 'Distance'])
clust_df['Classes_Label'] = train_lab_arr
pred_label = np.array(clust_df.Cluster)
orig_label = np.array(clust_df.Classes_Label)

print 'The Completeness Score is',completeness_score(orig_label, pred_label)
print 'The Homogeneity Score is',homogeneity_score(orig_label, pred_label)

The Completeness Score is 0.837786747868
The Homogeneity Score is 0.703868420359


### f 

Finally, using your cluster assignments as class labels, categorize each of the documents in the 20% set-aside data into each of the appropriate cluster. Your categorization should be based on Cosine similarity between each test document and each cluster centroids. Present your results in a separate file containing the obtained cluster label for each test document as well as Cosine similarities between each test document and each of the 5 clusters.

In [36]:

print df_TDIDF_test.shape
print center.shape

(500, 9328)
(5L, 9328L)


In [40]:
test_clusters = {}
rated = {}
pred_ratings = []

for i,array in enumerate(TDIDF_test):
    
    test_clusters[i] = []
    
    for i2, c in enumerate(center):
        
        cdistance =  1 - (np.dot(array,c) / (linalg.norm(array) * linalg.norm(c)) )
        test_clusters[i].append([i2,cdistance])


for key,value in test_clusters.items():

        value = sorted(value,key=lambda x:(-x[1],x[0]))
        pred = value[1][0]
        rated[key] = pred 
        pred_ratings.append(pred)

In [42]:
print len(pred_ratings)
print len(test_label_array)

500
2500


In [44]:
test_label_array = np.array(test_label[test_label.columns[0]])
newdf = pd.DataFrame()
newdf['Original Class'] = test_label_array
newdf['Predicted'] = pred_ratings

newdf.loc[newdf['Original Class'] == 0 ] = 'Windows'
newdf.loc[newdf['Original Class'] == 1 ] = 'Crypt'
newdf.loc[newdf['Original Class'] == 2 ] = 'Christian'
newdf.loc[newdf['Original Class'] == 3 ] = 'hockey'
newdf.loc[newdf['Original Class'] == 4 ] = 'forsale'

newdf.loc[newdf['Predicted'] == 0 ] = 'forsale - Predicted'
newdf.loc[newdf['Predicted'] == 1 ] = 'Christian - Predicted'
newdf.loc[newdf['Predicted'] == 2 ] = 'Windows - Predicted'
newdf.loc[newdf['Predicted'] == 3 ] = 'hockey - Predicted'
newdf.loc[newdf['Predicted'] == 4 ] = 'Crypt - Predicted'

In [50]:
newdf['Distance'] = test_clusters.items()
newdf.to_csv('Result_Test.csv')